# Update the image paths for the extracted features from Jan 29, 2018
March 23, 2018

In [30]:
import numpy as np
import pandas as pd
import imagehash

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
!pwd

/Users/ndbs/tree-logic/notebooks_gb_3_classes


In [29]:
!ls ..

LICENSE                data                   src
README.md              notebooks_gb_3_classes tree_photos
__pycache__            notebooks_initial      web


In [9]:
!ls ../src

__init__.py               image_classifier.py       remove_filename_spaces.sh
__pycache__               image_manager.py          scrape_images.py
dedup_images.py           model_eval.py


## Add parent directory to path to be able to import modules from a parallel directory
Solution from StackOverflow: https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im

In [15]:
#Solution to importing modules from a parallel directory from StackOverflow:
#https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [16]:
sys.path

['',
 '/usr/local/Cellar/apache-spark/2.2.0/libexec/python',
 '/Users/ndbs/tree-logic/notebooks_gb_3_classes',
 '/Users/ndbs/anaconda3/lib/python36.zip',
 '/Users/ndbs/anaconda3/lib/python3.6',
 '/Users/ndbs/anaconda3/lib/python3.6/lib-dynload',
 '/Users/ndbs/anaconda3/lib/python3.6/site-packages',
 '/Users/ndbs/anaconda3/lib/python3.6/site-packages/Mako-1.0.7-py3.6.egg',
 '/Users/ndbs/anaconda3/lib/python3.6/site-packages/aeosa',
 '/Users/ndbs/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/Users/ndbs/.ipython',
 '/Users/ndbs/tree-logic/src',
 '/Users/ndbs/tree-logic']

In [17]:
from src.image_manager import ImageManager
#Woo hoo, it worked!

## Create an ImageManager from my previously saved file database (saves as .csv's)
See `more_image_manager_and_scraper.ipynb` from Feb 5, 2018

In [2]:
!ls ../data

gb_confusion.png         image_log_20180205.csv   tree_image_info.csv
gb_model_jan30.pkl       image_syncs_20180204.csv tree_images.csv
image_log_20180204.csv   image_syncs_20180205.csv


In [18]:
manager = ImageManager('../tree_photos/', '../data/image_log_20180205.csv', '../data/image_syncs_20180205.csv')

In [19]:
manager.image_df.head()

,p_hash,filename,folder,time_added,time_verified
0,9bb759cb29b01a25,image_bigleaf_maple_57.png,acer_macrophyllum,1517842809809644000,1517842809809644000
1,99c20b3b74d53de0,image_bigleaf_maple_43.png,acer_macrophyllum,1517842809819089000,1517842809819089000
2,a1e1e0d4e646e765,image_big_leaf_maple_bark_48.png,acer_macrophyllum,1517842809827989000,1517842809827989000
3,abee94c19cb28b0e,image_acer_macrophylum_tree_49.png,acer_macrophyllum,1517842809836027000,1517842809836027000
4,d3c161631f2b4b4e,image_bigleaf_maple_94.png,acer_macrophyllum,1517842809842604000,1517842809842604000


In [20]:
manager.syncs_df

,time_started,time_completed,folders
0,1517842809802240000,1517842848258705000,"['acer_macrophyllum', 'betula_pendula', 'thuja..."
1,1517867697298852000,1517867781355441000,"['cedrus_libani', 'alnus_rubra', 'bad_images']"


In [22]:
!head '../data/image_log_20180205.csv'

|p_hash|filename|folder|time_added|time_verified
0|9bb759cb29b01a25|image_bigleaf_maple_57.png|acer_macrophyllum|2018-02-05 15:00:09.809644|2018-02-05 15:00:09.809644
1|99c20b3b74d53de0|image_bigleaf_maple_43.png|acer_macrophyllum|2018-02-05 15:00:09.819089|2018-02-05 15:00:09.819089
2|a1e1e0d4e646e765|image_big_leaf_maple_bark_48.png|acer_macrophyllum|2018-02-05 15:00:09.827989|2018-02-05 15:00:09.827989
3|abee94c19cb28b0e|image_acer_macrophylum_tree_49.png|acer_macrophyllum|2018-02-05 15:00:09.836027|2018-02-05 15:00:09.836027
4|d3c161631f2b4b4e|image_bigleaf_maple_94.png|acer_macrophyllum|2018-02-05 15:00:09.842604|2018-02-05 15:00:09.842604
5|c891b35fed643432|image_bigleaf_maple_80.png|acer_macrophyllum|2018-02-05 15:00:09.850563|2018-02-05 15:00:09.850563
6|b34194e34c0bf5d3|image_big_leaf_maple_bark_74.png|acer_macrophyllum|2018-02-05 15:00:09.858514|2018-02-05 15:00:09.858514
7|db7431cef4184de0|image_acer_macrophylum_tree_75.png|acer_macrophyllum|2018-02-05 15:00:09.866679|2018-0

### Ugh, why why are the timestamps converted to integers in the dataframes?!?!

## Load the previously extracted features
See `build_a_model.ipynb` from Jan 30, 2018

In [23]:
tree_features_df = pd.read_csv('../data/tree_images.csv', sep='|', index_col=0)
tree_features_df.head()

,p_hash,filename,species,tags,incv3_out_0,incv3_out_1,incv3_out_2,incv3_out_3,incv3_out_4,incv3_out_5,...,incv3_out_2038,incv3_out_2039,incv3_out_2040,incv3_out_2041,incv3_out_2042,incv3_out_2043,incv3_out_2044,incv3_out_2045,incv3_out_2046,incv3_out_2047
0,99a7465a1e99acb4,image_picea_sitchensis_27.png,picea_sitchensis,NaN,13.995308,1.316715,0.279829,67.839417,43.733177,38.206165,...,160.387985,1.176492,45.983730,10.465877,22.118896,3.226861,29.401524,0.486684,0.000000,15.634432
1,93d12f8e712ef068,image_picea_sitchensis_33.png,picea_sitchensis,NaN,7.449574,0.417233,0.447665,6.178374,2.092985,6.251201,...,2.649559,0.523397,0.392520,7.703988,13.842188,12.907540,1.112756,0.000000,5.436511,1.154409
2,cce3482eb991533b,image_picea_sitchensis_6.png,picea_sitchensis,NaN,0.708871,0.640444,1.305804,9.565918,15.157275,1.787687,...,15.114614,0.000000,0.066235,2.280773,8.082823,1.939496,0.041136,0.000000,0.000000,0.034437
3,c119991c4fcf1cda,image_picea_sitchensis_in_winter_12.png,picea_sitchensis,NaN,7.138457,1.225646,15.940742,42.644821,23.604031,13.818824,...,49.895081,0.000000,7.600242,3.260593,13.135204,12.249097,17.588070,0.000000,0.000000,0.000000
4,e3c3dab100fdc29c,image_sitka_spruce_59.png,picea_sitchensis,NaN,0.030419,2.616714,1.435360,52.145359,5.377345,5.631077,...,13.391465,21.970516,5.217728,8.282942,0.788009,0.591635,35.845409,0.000000,0.000000,2.780651


## Update `filename` and `species` columns in `tree_features_df` based on `filename` and `folder` columns in the ImageManager's dataframe

For every image (hash value) in `tree_features_df`, find the corresponding hash in `manager.image_df`, and update the columns accordingly.

In [26]:
manager.image_df['p_hash'].isin(tree_features_df['p_hash']).describe()

count      5760
unique        1
top       False
freq       5760
Name: p_hash, dtype: object

In [27]:
manager.image_df.loc[0,'p_hash']

array([[ True, False, False,  True,  True, False,  True,  True],
       [ True, False,  True,  True, False,  True,  True,  True],
       [False,  True, False,  True,  True, False, False,  True],
       [ True,  True, False, False,  True, False,  True,  True],
       [False, False,  True, False,  True, False, False,  True],
       [ True, False,  True,  True, False, False, False, False],
       [False, False, False,  True,  True, False,  True, False],
       [False, False,  True, False, False,  True, False,  True]], dtype=bool)

In [28]:
tree_features_df.loc[0,'p_hash']

'99a7465a1e99acb4'

### Crap, the hashes in ImageManager are imagehash objects, and those in tree_features_df are strings
So I guess let's convert them to hashes...

In [31]:
tree_features_df.loc[:,'p_hash'] = tree_features_df['p_hash'].apply(imagehash.hex_to_hash)
tree_features_df.loc[0,'p_hash']

array([[ True, False, False,  True,  True, False, False,  True],
       [ True, False,  True, False, False,  True,  True,  True],
       [False,  True, False, False, False,  True,  True, False],
       [False,  True, False,  True,  True, False,  True, False],
       [False, False, False,  True,  True,  True,  True, False],
       [ True, False, False,  True,  True, False, False,  True],
       [ True, False,  True, False,  True,  True, False, False],
       [ True, False,  True,  True, False,  True, False, False]], dtype=bool)

In [32]:
#And let's try this again:
manager.image_df['p_hash'].isin(tree_features_df['p_hash']).describe()

count      5760
unique        2
top       False
freq       4810
Name: p_hash, dtype: object

In [33]:
#Woo hoo, now we got some matches. Let's see if the numbers add up -- there should be 1014 Trues:
5760 - 4810

950

In [34]:
tree_features_df['p_hash'].describe()

count                 1014
unique                1014
top       ec39cc3871339837
freq                     1
Name: p_hash, dtype: object

### Crap! Why am I missing 64 of my original images?!

In [36]:
manager.image_df['p_hash'].isin(tree_features_df['p_hash']).head(20)

0     False
1      True
2      True
3     False
4      True
5      True
6      True
7     False
8      True
9      True
10    False
11     True
12     True
13     True
14     True
15     True
16    False
17    False
18    False
19     True
Name: p_hash, dtype: bool

In [38]:
manager.image_df.loc[1,:] #This image should be in both dataframes

p_hash                     99c20b3b74d53de0
filename         image_bigleaf_maple_43.png
folder                    acer_macrophyllum
time_added              1517842809819089000
time_verified           1517842809819089000
Name: 1, dtype: object

In [39]:
tree_features_df[tree_features_df['p_hash'] == manager.image_df.loc[1,'p_hash']] #Yes, it's in both, with the same name

,p_hash,filename,species,tags,incv3_out_0,incv3_out_1,incv3_out_2,incv3_out_3,incv3_out_4,incv3_out_5,...,incv3_out_2038,incv3_out_2039,incv3_out_2040,incv3_out_2041,incv3_out_2042,incv3_out_2043,incv3_out_2044,incv3_out_2045,incv3_out_2046,incv3_out_2047
645,99c20b3b74d53de0,image_bigleaf_maple_43.png,acer_macrophylum,NaN,10.294667,1.889718,24.958405,86.076622,0.0,13.88846,...,2.308625,4.051409,15.949759,7.017689,3.985941,1.784329,69.541832,0.270866,0.0,5.083283


In [40]:
tree_features_df['p_hash'].isin(manager.image_df['p_hash']).describe()
#Try indexing the other way -- still only 950 matches

count     1014
unique       2
top       True
freq       950
Name: p_hash, dtype: object

In [41]:
tree_features_df[~tree_features_df['p_hash'].isin(manager.image_df['p_hash'])] #Let's look at what's missing:

,p_hash,filename,species,tags,incv3_out_0,incv3_out_1,incv3_out_2,incv3_out_3,incv3_out_4,incv3_out_5,...,incv3_out_2038,incv3_out_2039,incv3_out_2040,incv3_out_2041,incv3_out_2042,incv3_out_2043,incv3_out_2044,incv3_out_2045,incv3_out_2046,incv3_out_2047
6,ff6042c59f3870e2,image_sitka_spruce_71.png,picea_sitchensis,NaN,3.729056,0.905666,7.141092,55.251953,5.411305,11.176881,...,1.971251,2.367423,13.462605,0.506278,5.015826,0.000000,37.992500,0.000000,0.000000,0.017231
23,e6e6c9791516c2c3,image_picea_sitchensis_in_winter_11.png,picea_sitchensis,NaN,0.000000,2.008564,0.034550,73.674919,13.123478,6.481175,...,45.059196,2.851241,18.120329,1.649355,0.846053,4.039718,49.071014,0.000000,0.000000,0.000000
30,a3cc1eb06c9268bf,image_sitka_spruce_cones_67.png,picea_sitchensis,NaN,4.241492,0.594667,10.149830,68.010300,6.601846,17.952438,...,8.778487,10.241682,3.941283,5.683080,6.083897,0.000000,45.048485,0.000000,0.000000,6.027557
44,d150e161f60be95d,image_picea_sitchensis_35.png,picea_sitchensis,NaN,3.673640,0.504717,20.620829,13.874782,4.447742,13.795573,...,10.101775,0.085565,0.114732,3.356215,13.354259,8.211310,1.402413,0.800033,0.006432,1.478342
85,f969841786b1c6b5,image_picea_sitchensis_87.png,picea_sitchensis,NaN,8.863665,16.567057,0.000000,113.286438,65.563690,40.489342,...,197.107941,1.142475,173.990662,0.100046,4.880579,13.690717,4.626838,0.627302,0.171533,20.922581
96,d09a46d13fa83f46,image_sitka_spruce_cones_13.png,picea_sitchensis,NaN,6.770698,9.003544,4.424788,12.767536,15.125897,24.266603,...,38.625927,1.533867,10.232586,4.473899,11.273945,23.888660,0.000000,0.000000,0.000000,0.295313
98,80f929eef075634a,image_sitka_spruce_cones_5.png,picea_sitchensis,NaN,10.563470,1.933169,20.351706,56.973434,3.866005,22.816250,...,15.392019,1.859062,11.294960,2.817833,6.578317,2.314414,37.831131,0.000000,0.000000,0.446455
101,fffe00877f940092,image_picea_sitchensis_in_winter_70.png,picea_sitchensis,NaN,10.848274,1.375554,27.521946,22.071404,0.448212,22.874340,...,11.961102,0.000000,0.021970,10.924964,9.276912,9.060322,18.617376,0.000000,0.228338,0.000000
139,af7cd0a335d243c4,image_sitka_spruce_cones_28.png,picea_sitchensis,NaN,18.092546,16.393559,0.670490,131.969284,27.032993,40.941666,...,128.238953,12.098897,123.055511,0.000000,28.015902,1.077125,65.696526,0.000000,0.000000,12.710382
160,ffc8406bdb369422,image_picea_sitchensis_54.png,picea_sitchensis,NaN,10.870422,7.001470,25.653788,20.377882,4.681588,31.870710,...,6.042257,3.319450,1.570931,12.889059,2.060805,0.000000,13.069489,0.000000,0.000000,0.507192


In [42]:
manager.image_df[manager.image_df['filename'] == 'image_sitka_spruce_71.png']
#ff6042c59f3870e2 -- original hash
#fe6046c59f3870e2 -- new hash, off by 1

,p_hash,filename,folder,time_added,time_verified
2964,fe6046c59f3870e2,image_sitka_spruce_71.png,picea_sitchensis,1517842834196173000,1517842834196173000


In [43]:
manager.image_df[manager.image_df['filename'] == 'image_picea_sitchensis_in_winter_11.png']
#e6e6c9791516c2c3 -- original hash
#e6e6c9791516c2e2 -- new hash, off by 1

,p_hash,filename,folder,time_added,time_verified
3000,e6e6c9791516c2e2,image_picea_sitchensis_in_winter_11.png,picea_sitchensis,1517842834493497000,1517842834493497000


In [44]:
imagehash.hex_to_hash('ff6042c59f3870e2') - imagehash.hex_to_hash('fe6046c59f3870e2')

2

In [45]:
imagehash.hex_to_hash('e6e6c9791516c2c3') - imagehash.hex_to_hash('e6e6c9791516c2e2')

2

### Ok, so (based on a sample size of 2) it looks like the hashes "almost" match, but not quite. WTF?

## See if I can match by by filename...
Fewer matches, but perhaps by combining I can find most of the images...

In [46]:
tree_features_df['filename'].isin(manager.image_df['filename']).describe()

count     1014
unique       2
top       True
freq       885
Name: filename, dtype: object

In [47]:
#Find all images in original features df where neither the hash nor the filename match anything in the new database
tree_features_df[~(tree_features_df['p_hash'].isin(manager.image_df['p_hash']) | tree_features_df['filename'].isin(manager.image_df['filename']))]

,p_hash,filename,species,tags,incv3_out_0,incv3_out_1,incv3_out_2,incv3_out_3,incv3_out_4,incv3_out_5,...,incv3_out_2038,incv3_out_2039,incv3_out_2040,incv3_out_2041,incv3_out_2042,incv3_out_2043,incv3_out_2044,incv3_out_2045,incv3_out_2046,incv3_out_2047
193,c4196ab76b923639,image_sitka_spruce_cones_24.png,picea_sitchensis,NaN,9.484322,0.456599,17.090580,33.145550,7.634374,37.828159,...,3.752788,1.529060,4.443648,7.727086,2.551901,1.993320,21.856192,0.0,2.79655,0.000000
524,a635ec05f863df01,image_platanus_acerifolia_in_winter_70.png,platanus_acerifolia,NaN,13.074394,1.356100,1.736537,89.516685,6.171581,27.867926,...,56.403465,21.606693,28.985575,1.070458,18.911673,1.787604,58.742775,0.0,0.00000,0.916422
586,db66218ebc796aa0,image_london_plane_17.png,platanus_acerifolia,NaN,5.451069,2.250515,2.184188,82.828766,8.848424,22.939686,...,25.899662,0.000000,22.401608,0.462651,10.309099,0.000000,41.468472,0.0,0.00000,1.180786
597,e1f4981cbd78b123,image_platanus_acerifolia_12.png,platanus_acerifolia,NaN,7.175202,4.094048,0.099406,20.957792,11.252434,16.618130,...,10.776367,1.373431,4.903385,16.547131,16.293879,1.170623,20.847250,0.0,0.00000,3.254873
608,f39ce1daa6691861,image_platanus_acerifolia_fruit_98.png,platanus_acerifolia,NaN,0.404047,0.528681,1.291309,91.310089,2.383657,22.805372,...,4.871082,15.679230,29.035728,4.291573,0.000000,0.000000,63.750210,0.0,0.00000,16.916330
624,fff16d7888408578,image_platanus_acerifolia_fruit_89.png,platanus_acerifolia,NaN,10.286492,9.711628,13.248781,38.256615,26.349409,32.878189,...,42.892338,0.000000,23.105974,9.367880,7.283285,3.244074,10.335363,0.0,0.00000,0.000000
923,88557b1f5ec911b1,image_big_leaf_maple_bark_2.png,acer_macrophylum,NaN,20.104141,1.937378,24.418465,19.040472,2.304081,31.558853,...,6.873886,2.210545,2.524226,18.855762,10.737486,3.363316,1.183610,0.0,13.39602,0.000000


### Ok great, so there are only 7 of the original images that don't have a matching hash or filename

Is it true that only 7 of my original files got deleted when I did the deduplication?

For each of these, I can try to find the closest matching hash, somehow...

Also, I should verify that if the filenames match but the hashes don't, then the hashes are at least close to each other.